In [6]:
import numpy as np
import h5py
import pandas as pd

In [7]:
data = {}
targets = {}
files = open('files', 'r')
lines = files.readlines()
count = {}
for line in lines:
    fileName, target = line.strip().split(";")
    with h5py.File(fileName, 'r') as f:
        for k, v in f.items():
            tmp=np.array(v).transpose()
            if not target in data.keys():
                data[target]=tmp
                count[target] = tmp.shape[0]
            else:
                data[target] = np.concatenate((data[target], tmp))
                count[target]+= tmp.shape[0]
totalCount = 0
i=0
targetsColumn = None
for k, v in count.items():
    targets[k]=np.ones(v, dtype=int)*int(i)
    totalCount += v
    if i==0:
        targetsColumn = targets[k]
    else:
        targetsColumn = np.concatenate((targetsColumn, targets[k]))
    i += 1
targetsColumn=targetsColumn.reshape(targetsColumn.shape[0], 1)
dataSet = None
for k, v in data.items():
    if dataSet is None:
        dataSet = data[k]
    else:
        dataSet = np.concatenate((dataSet, data[k]))
dataSet = np.concatenate((dataSet, targetsColumn), axis=1)
dataSet

array([[10856,  9761,  8767, ..., 11280,  7453,     0],
       [10883,  9767,  8845, ..., 10465,  7143,     0],
       [10893,  9787,  8797, ..., 10563,  7026,     0],
       ...,
       [10026,  9285,  9514, ..., 14579,  8932,     2],
       [10004,  9281,  9451, ..., 16861,  9970,     2],
       [ 9875,  8973,  8637, ..., 12962,  8935,     2]])

In [8]:
df = pd.DataFrame(data=dataSet, columns=["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2", "Target"])
df['Target'] = df['Target'].replace({0:"coca", 1:"nubes", 2:"cana"})
df

Costal  Blue  Green   Red    NIR  SWIR1  SWIR2 Target
0       10856  9761   8767  7646  17400  11280   7453   coca
1       10883  9767   8845  7616  17759  10465   7143   coca
2       10893  9787   8797  7574  17908  10563   7026   coca
3       10844  9813   8808  7617  17533  10779   7173   coca
4       10945  9848   9001  7699  21179  12428   7952   coca
...       ...   ...    ...   ...    ...    ...    ...    ...
45402   10053  9231   8983  8140  21430  13972   9350   cana
45403    9465  8638   8662  8334  21553  12664   7129   cana
45404   10026  9285   9514  8931  21104  14579   8932   cana
45405   10004  9281   9451  8706  23822  16861   9970   cana
45406    9875  8973   8637  7995  15796  12962   8935   cana

[45407 rows x 8 columns]

In [9]:
#Shuffle dataset mix
df = df.sample(frac=1).reset_index(drop=True)
df

Costal   Blue  Green    Red    NIR  SWIR1  SWIR2 Target
0       10025   9023   8164   7090  17569  10246   6962   coca
1        9134   8315   7841   6784  20478  13155   8595   cana
2       11710  10929  10928  10763  12401  11752   9629   cana
3       11077   9950   8928   7876  18640  11161   7351   coca
4       10127   9460   9240   9377  18019  17781  12277   cana
...       ...    ...    ...    ...    ...    ...    ...    ...
45402   11131  10046   9342   8320  18738  14174   9533   coca
45403    9685   8747   8174   7192  19462  11142   7789   cana
45404   10112   9169   8768   8045  18230  11414   8068   cana
45405   10626   9614   8786   7805  16594  10449   7369   coca
45406   11683  11294  11519  12883  12966  15188  13264   cana

[45407 rows x 8 columns]

In [ ]:
#Creating the sub sets : Training set 70%, Test set 20%, Validation set 10%

In [12]:
from sklearn.model_selection import train_test_split

features = ["Costal", "Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]
labels = ["coca", "cana", "nubes"]

X=df.loc[:, features]
y=df.loc[:, "Target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=109)


In [15]:
#Fitting parameters and validating it with validation set
from sklearn import svm

vdf = df.sample(frac=0.1).reset_index(drop=True)

X_v = vdf.loc[:, features]
y_v = vdf.loc[:, "Target"]

X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(X_v, y_v, test_size=0.3, random_state=109)

classifier = svm.SVC(kernel='linear')
classifier.fit(X_v_train, y_v_train)

y_v_pred = classifier.predict(X_v_test)

array(['coca', 'coca', 'nubes', ..., 'nubes', 'nubes', 'cana'],
      dtype=object)

In [18]:
from sklearn import metrics
metrics.confusion_matrix(y_v_test, y_v_pred, labels)

(1363,)

In [19]:
XX=vdf.loc[:, features]
yy=classifier.predict(XX)

In [25]:
from sklearn import metrics
print(metrics.confusion_matrix(y_v, yy, labels))
print(metrics.precision_recall_fscore_support(y_v, yy))

[[1531   59    0]
 [  38 1653    0]
 [   0    0 1260]]
(array([0.96553738, 0.97578075, 1.        ]), array([0.97752809, 0.96289308, 1.        ]), array([0.97149574, 0.96929408, 1.        ]), array([1691, 1590, 1260]))


In [21]:
from sklearn import metrics
print("Accuracy: ", metrics.accuracy_score(y_v_test, y_v_pred))
#The classifier works fine on validation set, now it's time to training over all the data set and
#Check if results are the same

Accuracy:  0.983125458547322


In [22]:
classifier = svm.SVC(kernel='linear')
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [26]:
#Prediction and metrics of the classifier predicts to the entire test set
y_pred = classifier.predict(X_test)

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Accuracy:  0.979299713719445
